In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass

from pathlib import Path
from transformers import RobertaTokenizer, RobertaModel

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from PIL import Image

from transformers import ViTImageProcessor, ViTForImageClassification, ViTFeatureExtractor
import os
from going_modular import engine

import pandas as pd
import numpy as np
import itertools

import albumentations as A
from albumentations.pytorch import ToTensorV2



In [14]:
@dataclass
class ModelArgs:
    device = 'cuda'
    vis_embd_out = 768
    text_embd_out = 512

In [ ]:
#Vision Encoder (CLiP-Vision Encoder)
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_vision = clip.vision_model

#Freezing the weights

for params in clip_vision.parameters():
    params = False

In [ ]:
clip_vision.vision_model

In [ ]:
#Language Decoder

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.3")
lang_model = AutoModelForCausalLM.from_pretrained("lmsys/vicuna-7b-v1.3")

for params in lang_model.parameters():
    params = False

In [17]:
#Projector

class Projector(nn.Module):
    def __init__(self):
        
        super()._init_()
        
        self.linear_layer = nn.Linear(in_features=ModelArgs.vis_embd_out, out_features=ModelArgs.text_embd_out, device=ModelArgs.device)
        
    def forward(self, x):
        out = self.linear_layer(x)
        return out

In [18]:
class Llava(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        
        self.projector = Projector()
        
        
    def forward(self, text, img):
        vis_out = clip_vision(img)
        proj_out = self.projector(vis_out)
        combined_out = lang_model(torch.cat([proj_out, text], axis=0))
        return combined_out
        

In [ ]:
llava = Llava()